In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import seaborn as sns

from patsy import dmatrices
from scipy import stats
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from scipy.stats import spearmanr

In [2]:
path_file = "C:/Users/Grios/Desktop/pipe/Proyecto/Correcciones/Apendice_G.xlsx"
dta = pd.read_excel(path_file)
df = pd.DataFrame(dta)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3630 entries, 0 to 3629
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ARRIENDO    3630 non-null   float64
 1   SERVINT     3630 non-null   float64
 2   PUBLICI     3630 non-null   float64
 3   TRANSP      3630 non-null   float64
 4   SERVEXT2    3630 non-null   float64
 5   TOTAL_REM2  3630 non-null   float64
 6   TOTPERSO    3630 non-null   float64
 7   INVPRO      3630 non-null   float64
 8   CTO         3630 non-null   float64
 9   VENTA       3630 non-null   float64
dtypes: float64(10)
memory usage: 283.7 KB


In [3]:
dta.corr('spearman')

,ARRIENDO,SERVINT,PUBLICI,TRANSP,SERVEXT2,TOTAL_REM2,TOTPERSO,INVPRO,CTO,VENTA
ARRIENDO,1.000000,0.408130,0.128876,0.297601,0.318003,0.375657,0.197813,0.358995,0.376857,0.187275
SERVINT,0.408130,1.000000,0.051596,0.470604,0.588970,0.768410,0.596929,0.555551,0.705526,0.504304
PUBLICI,0.128876,0.051596,1.000000,0.243668,0.132320,0.115796,-0.054445,0.181687,0.126688,-0.061947
TRANSP,0.297601,0.470604,0.243668,1.000000,0.453074,0.514242,0.333991,0.373544,0.492568,0.288674
SERVEXT2,0.318003,0.588970,0.132320,0.453074,1.000000,0.580950,0.418131,0.463063,0.544026,0.369664
TOTAL_REM2,0.375657,0.768410,0.115796,0.514242,0.580950,1.000000,0.755694,0.579513,0.745206,0.552772
TOTPERSO,0.197813,0.596929,-0.054445,0.333991,0.418131,0.755694,1.000000,0.403675,0.597863,0.763617
INVPRO,0.358995,0.555551,0.181687,0.373544,0.463063,0.579513,0.403675,1.000000,0.673051,0.456992
CTO,0.376857,0.705526,0.126688,0.492568,0.544026,0.745206,0.597863,0.673051,1.000000,0.786450
VENTA,0.187275,0.504304,-0.061947,0.288674,0.369664,0.552772,0.763617,0.456992,0.786450,1.000000


In [4]:
#SPEARMAN
Fil = dta['PUBLICI']
Col = dta['TRANSP']

coef, p = spearmanr(Fil, Col)
print('Coeficiente de correlacion de Spearman: %.3f' % coef)

#Interpretar significancia
alpha = 0.05  #95% de probalidad de acierto de que las pruebas no esten correlacionadas
if p > alpha:
    print('Las pruebas estan no relacionadas (fallo en rechazar la hipotesis) p=%.3f' % p)
else:
    print('Las pruebas estan relacionadas (rechaza la hipotesis) p=%.3f' % p)

Coeficiente de correlacion de Spearman: 0.244
Las pruebas estan relacionadas (rechaza la hipotesis) p=0.000


In [5]:
#SPEARMAN
Fil = dta['TOTAL_REM2']
Col = dta['TOTPERSO']

coef, p = spearmanr(Fil, Col)
print('Coeficiente de correlacion de Spearman: %.3f' % coef)

#Interpretar significancia
alpha = 0.05  #95% de probalidad de acierto de que las pruebas no esten correlacionadas
if p > alpha:
    print('Las pruebas estan no relacionadas (fallo en rechazar la hipotesis) p=%.3f' % p)
else:
    print('Las pruebas estan relacionadas (rechaza la hipotesis) p=%.3f' % p)

Coeficiente de correlacion de Spearman: 0.756
Las pruebas estan relacionadas (rechaza la hipotesis) p=0.000


In [6]:
dta.drop(['PUBLICI','TOTAL_REM2','INVPRO','SERVEXT2'], axis=1, inplace=True)
dta

,ARRIENDO,SERVINT,TRANSP,TOTPERSO,CTO,VENTA
0,0.006629,0.005813,0.008565,0.001605,0.016319,0.420915
1,0.000261,0.000262,0.001022,0.001783,0.003807,0.455799
2,0.006629,0.005813,0.008565,0.001783,0.016319,0.284335
3,0.000104,0.000285,0.000121,0.001783,0.003423,0.447846
4,0.006629,0.005813,0.008565,0.001961,0.016319,0.407667
...,...,...,...,...,...,...
3625,0.000127,0.000866,0.000221,0.001783,0.000763,0.322389
3626,0.000010,0.000954,0.001325,0.002496,0.002143,0.409969
3627,0.000121,0.000537,0.008565,0.001783,0.002049,0.395980
3628,0.006629,0.004568,0.002951,0.026391,0.012081,0.584510


In [7]:
X = pd.DataFrame(dta.iloc[:,0:5].values)
print(X)
X = sm.add_constant(X)
y = dta.iloc[:,[5]]
print(y)

             0         1         2         3         4
0     0.006629  0.005813  0.008565  0.001605  0.016319
1     0.000261  0.000262  0.001022  0.001783  0.003807
2     0.006629  0.005813  0.008565  0.001783  0.016319
3     0.000104  0.000285  0.000121  0.001783  0.003423
4     0.006629  0.005813  0.008565  0.001961  0.016319
...        ...       ...       ...       ...       ...
3625  0.000127  0.000866  0.000221  0.001783  0.000763
3626  0.000010  0.000954  0.001325  0.002496  0.002143
3627  0.000121  0.000537  0.008565  0.001783  0.002049
3628  0.006629  0.004568  0.002951  0.026391  0.012081
3629  0.000241  0.000285  0.000086  0.001605  0.000999

[3630 rows x 5 columns]
         VENTA
0     0.420915
1     0.455799
2     0.284335
3     0.447846
4     0.407667
...        ...
3625  0.322389
3626  0.409969
3627  0.395980
3628  0.584510
3629  0.326728

[3630 rows x 1 columns]


In [8]:
mod_ols = sm.OLS(y, X)
res_ols = mod_ols.fit()
print(res_ols.summary())

y_pred_o = res_ols.predict(X)
error_o = np.sqrt(mean_squared_error(y, y_pred_o))
print('Error: ', error_o)
print('R2: ', res_ols.rsquared)

                            OLS Regression Results                            
Dep. Variable:                  VENTA   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     591.6
Date:                Mon, 11 Jan 2021   Prob (F-statistic):               0.00
Time:                        14:34:54   Log-Likelihood:                 3806.6
No. Observations:                3630   AIC:                            -7601.
Df Residuals:                    3624   BIC:                            -7564.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4663      0.002    278.218      0.0

In [9]:
reg = LinearRegression()
regR = reg.fit(X, y)

y_pred = regR.predict(X)
error = np.sqrt(mean_squared_error(y, y_pred))
r2 = reg.score(X, y)

In [10]:
print("El error es: ", error)
print("El valor de r^2 es: ", r2)
print("Los coeficientes son: ", reg.coef_)

El error es:  0.084787918340191
El valor de r^2 es:  0.44940113650994296
Los coeficientes son:  [[ 0.         -2.56617086 -0.16929412 -1.38306497  3.27714394  1.41416233]]


### Apendice_h

In [11]:
path_file = "C:/Users/Grios/Desktop/pipe/Proyecto/Correcciones/Apendice_F.xlsx"
dta_co = pd.read_excel(path_file)
df_co = pd.DataFrame(dta_co)
df_co.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3630 entries, 0 to 3629
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   comp1   3630 non-null   float64
 1   comp2   3630 non-null   float64
 2   VENTA   3630 non-null   float64
dtypes: float64(3)
memory usage: 85.1 KB


In [12]:
dta_co

,comp1,comp2,VENTA
0,0.007035,0.000527,0.420915
1,-0.012700,0.003650,0.455799
2,0.007035,0.000527,0.284335
3,-0.015216,-0.000089,0.447846
4,0.007035,0.000527,0.407667
...,...,...,...
3625,-0.019602,-0.000039,0.322389
3626,-0.018018,0.000140,0.409969
3627,-0.014546,-0.005195,0.395980
3628,-0.004313,-0.003408,0.584510


In [13]:
X = pd.DataFrame(dta_co.iloc[:,0:2].values)
print(X)
X = sm.add_constant(X)
y = dta_co.iloc[:,[2]]
print(y)

             0         1
0     0.007035  0.000527
1    -0.012700  0.003650
2     0.007035  0.000527
3    -0.015216 -0.000089
4     0.007035  0.000527
...        ...       ...
3625 -0.019602 -0.000039
3626 -0.018018  0.000140
3627 -0.014546 -0.005195
3628 -0.004313 -0.003408
3629 -0.019620  0.000071

[3630 rows x 2 columns]
         VENTA
0     0.420915
1     0.455799
2     0.284335
3     0.447846
4     0.407667
...        ...
3625  0.322389
3626  0.409969
3627  0.395980
3628  0.584510
3629  0.326728

[3630 rows x 1 columns]


In [14]:
mod_ols = sm.OLS(y, X)
res_ols = mod_ols.fit()
print(res_ols.summary())

y_pred_o = res_ols.predict(X)
error_o = np.sqrt(mean_squared_error(y, y_pred_o))
print('Error: ', error_o)
print('R2: ', res_ols.rsquared)

                            OLS Regression Results                            
Dep. Variable:                  VENTA   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                     1131.
Date:                Mon, 11 Jan 2021   Prob (F-statistic):               0.00
Time:                        14:34:55   Log-Likelihood:                 3603.1
No. Observations:                3630   AIC:                            -7200.
Df Residuals:                    3627   BIC:                            -7182.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4956      0.001    332.831      0.0

In [15]:
reg = LinearRegression()
reg = reg.fit(X, y)

y_pred = reg.predict(X)
error = np.sqrt(mean_squared_error(y, y_pred))
r2 = reg.score(X, y)

In [16]:
print("El error es: ", error)
print("El valor de r^2 es: ", r2)
print("Los coeficientes son: ", reg.coef_)

El error es:  0.08967707968225035
El valor de r^2 es:  0.38407152847810244
Los coeficientes son:  [[ 0.          1.66718015 -1.03723989]]
